# 1.清单计算

In [1]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd





#一、导入指导价

#二、打开对方清单

def suanqingdan():
    suanqingdan=0
    global url,qing,qing2,shijian0
    jia_df=pd.read_excel('转贴现买断价格计算/jia.xls',encoding='GB18030')

    shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
    shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式

#2.1输入交易日
    jin=input('请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:')
    if jin!='':
        shijian=datetime.datetime.strptime(jin , "%Y-%m-%d")
    jia_df['生效截至日'] = pd.to_datetime(jia_df['生效截至日'])#.astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
    jia_df=jia_df[((jia_df['生效截至日']>=shijian) | (jia_df['生效截至日']==''))&(jia_df['银行分类']!='资管、证券、金融公司')]

    jia_df= jia_df.reset_index(drop=True)    #重新定义索引
    qixian1=jia_df.columns[2].split('至')
    qixian2=jia_df.columns[3].split('至')
    qixian3=jia_df.columns[4].split('至')
    qixian4=jia_df.columns[5].split('至')
    qixian5=jia_df.columns[6].split('至')

    qixian1[0] = datetime.datetime.strptime(qixian1[0] , "%Y-%m-%d")
    qixian1[1] = datetime.datetime.strptime(qixian1[1] , "%Y-%m-%d")
    qixian2[0] = datetime.datetime.strptime(qixian2[0] , "%Y-%m-%d")
    qixian2[1] = datetime.datetime.strptime(qixian2[1] , "%Y-%m-%d")
    qixian3[0] = datetime.datetime.strptime(qixian3[0] , "%Y-%m-%d")
    qixian3[1] = datetime.datetime.strptime(qixian3[1] , "%Y-%m-%d")
    qixian4[0] = datetime.datetime.strptime(qixian4[0] , "%Y-%m-%d")
    qixian4[1] = datetime.datetime.strptime(qixian4[1] , "%Y-%m-%d")
    qixian5[0] = datetime.datetime.strptime(qixian5[0] , "%Y-%m-%d")
    qixian5[1] = datetime.datetime.strptime(qixian5[1] , "%Y-%m-%d")


#2.2确认列名，到时候可以删除
    qing=pd.read_excel(url)
    print(qing.head())
#二、换列标题
    qingcolumn=qing.columns.values.tolist()  
    qingcolumn_df=pd.read_excel('转贴现买断价格计算/lieming.csv',encoding='GB18030')
    for i in range(0,len(qingcolumn)):
        for j in range(0,len(qingcolumn_df)):
            zhaop2= re.search(qingcolumn_df.loc[j,'原列名'] ,qingcolumn[i])
 

            if zhaop2:
                qing.rename(columns={qingcolumn[i]:qingcolumn_df.loc[j,'新列名']}, inplace=True)
                break

    if '贴现行' not in qing.columns.values.tolist():
        qing['贴现行']=''


#三、筛选列，计算清单
    qing=qing[['票号','票面金额','出票人','承兑行','贴现行','出票日','到期日']]
    qing= qing.reset_index(drop=True)    #重新定义索引
    for i in range(0,len(qing)):
        qing.loc[i,'承兑行号']=str(qing.loc[i,'票号'])[1:13]
        
        
    qing['到期日2'] = pd.to_datetime(qing['到期日'])   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
    
    qing= qing.reset_index(drop=True)    #重新定义索引
    qing['承兑行号']=qing['承兑行号'].astype(str)
    
  #  qing['出票日']=qing['出票日'].astype(str)
    qing['出票日'] = pd.to_datetime(qing['出票日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
    qing['到期日'] = pd.to_datetime(qing['到期日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
        
  #  qing['到期日']=qing['到期日'].astype(str)

    for i in range (0,len(qing)):     
         if (qing.loc[i,'到期日2']>=qixian1[0]) and (qing.loc[i,'到期日2']<=qixian1[1]) :
           for j in range (0,len(jia_df)):
               if (jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']) and (float(jia_df.loc[j,jia_df.columns[2]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]+0.10
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]
                       qing.loc[i,'小票票据']=0
                    

                   qing.loc[i,'指导期限']=jia_df.columns[2]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
               
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')

                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian2[0]) and (qing.loc[i,'到期日2']<=qixian2[1]) :

           for j in range (0,len(jia_df)):

               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']    and (float(jia_df.loc[j,jia_df.columns[3]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]+0.10
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]
                       qing.loc[i,'小票票据']=0
                    
                   qing.loc[i,'指导期限']=jia_df.columns[3]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]
                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian3[0]) and (qing.loc[i,'到期日2']<=qixian3[1])  :  #

           for j in range (0,len(jia_df)):
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']  and (float(jia_df.loc[j,jia_df.columns[4]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]+0.10
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]
                       qing.loc[i,'小票票据']=0

                   qing.loc[i,'指导期限']=jia_df.columns[4]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian4[0]) and (qing.loc[i,'到期日2']<=qixian4[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[5]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]+0.10
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[5]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1




         elif (qing.loc[i,'到期日2']>=qixian5[0]) and (qing.loc[i,'到期日2']<=qixian5[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[6]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]+0.10
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[6]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1
                   
    if suanqingdan<=0:
        print('整个清单都不能收！或者请清单检查格式。提示：1.出票日、到期日要是日期格式，2.是否有其他“承兑”、“付款”之类的列名存在，系统误认为是错误的“承兑行”列。')
        del qing['到期日2']
        qing2=qing
        
    else:
        qing['加权利率']=qing.loc[:,'ji2'].sum()/qing.loc[:,'ji1'].sum()
        del qing['到期日2']



        qing=qing[['票号','票面金额','出票人','承兑行','承兑行号','出票日','到期日','交易日','银行分类','邮储指导价','小票票据','指导期限','加权利率','贴现行','可用额度','我行评级','生效截至日','授信有效','期限','到期月']]
#三、将算好的清单写入excel
        qing2=qing.replace(np.nan,99)
     
        qing2=qing2[qing2['邮储指导价'] != 99]  #排除NAN
        chengdui=qing2['票面金额'].groupby(qing2['银行分类']).sum()/10000
        print('  ')

        zongjin=qing2['票面金额'].sum()/10000
        print('总金额：%s'%zongjin)
        print('  清单分类如下：')

        print(chengdui)
        print('  ')
        yuefen=qing2['票面金额'].groupby(qing2['到期月']).sum()/10000
        print(yuefen)
        print('  ')
        qixian=qing2['期限'].mean()
        print(qixian)
        print('  ')
        print('提示：票据持有方式选：持有！申报清单总金额和张数是否和对方实际发来的一致？承兑行是否有错别字？')

    
    





#四、制作导
        qing2['导汇票']=1
        qing2['导出票人账号']=''
        qing2['导收款人名称']=''
        qing2['导1']=''
        qing2['导2']=''
        qing2['导3']=''
        qing2['同城']=1
        qing2=qing2[['导汇票','票号','票面金额','出票日','到期日','出票人','导出票人账号','承兑行号','承兑行','导收款人名称','导1','导2','导3','承兑行号','同城','交易日','期限']]

        qing2=qing2.fillna('')  #把日期列的nan 数值用空值填充，统一空值

        print(qing2)
        qing2=qing2[(qing2['票号'] !='')]
        qing2= qing2.reset_index(drop=True)    #重新定义索引
        qing=qing.fillna('')  #把日期列的nan 数值用“NAN”填充，统一空值


        qing=qing[(qing['票号'] !='')]
        qing= qing.reset_index(drop=True)    #重新定义索引
        
        
def heimingdan():
    global qing,qing2,shijian0
    hei1=pd.read_excel('转贴现买断价格计算/系统黑名单.xls',encoding='GB18030')
  #  hei2=pd.read_excel('转贴现买断价格计算/系统黑名单企业.xls',encoding='GB18030')           
    
    for i in range(0,len(hei1)):
        if '/' in  hei1.loc[i,'银行黑名单']:      
            erhei= hei1.loc[i,'银行黑名单'].split('/')
            for j in range(0,len(qing)):
                if (erhei[0] in qing.loc[j,'承兑行']) and (erhei[1] in qing.loc[j,'承兑行']):
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
            
                   # break
            
            
            
        else:
            for j in range(0,len(qing)):
                if hei1.loc[i,'银行黑名单'] in qing.loc[j,'承兑行']:
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                  #  break
    url3='临时清单/'+shijian0+duifang+'导入.xls'
    writer = pd.ExcelWriter(url3)
    qing2.to_excel(writer,'Sheet1')
    writer.save()

    url2='临时清单/'+shijian0+duifang+'算加权.xls'
    writer = pd.ExcelWriter(url2)
    qing.to_excel(writer,'Sheet1')
    writer.save()

        
for i in range(0,1):
    print('\n')
    duifang=input('请输入第%s个文件名：'%i)
 #   duifang=''+duifang|
    url=('临时清单/%s'%duifang)
    if duifang=='':
        continue
    else:
        
        suanqingdan()
        heimingdan() 




请输入第0个文件名：卖舟山定海农商.xlsx
请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:
                                 票号    票面金额       出票日期        到期日  \
385  130939720530020200806696388342  300000 2020-08-06 2020-12-06   

                承兑人         出票人          收款人                  贴现行  
385  兴业银行股份有限公司晋江支行  利郎（中国）有限公司  江苏扬子豚服饰有限公司  中国邮政储蓄银行股份有限公司杭州市分行  
  
总金额：30.0
  清单分类如下：
银行分类
国股（授信银行贴）    30.0
Name: 票面金额, dtype: float64
  
到期月
2020-12    30.0
Name: 票面金额, dtype: float64
  
27.0
  
提示：票据持有方式选：持有！申报清单总金额和张数是否和对方实际发来的一致？承兑行是否有错别字？
   导汇票                              票号    票面金额         出票日         到期日  \
0    1  130939720530020200806696388342  300000  2020-08-06  2020-12-06   

          出票人 导出票人账号          承兑行号             承兑行 导收款人名称 导1 导2 导3  \
0  利郎（中国）有限公司         309397205300  兴业银行股份有限公司晋江支行                   

           承兑行号  同城         交易日    期限  
0  309397205300   1  2020-11-09  27.0  


# 重要交易对手同业存单

In [3]:
# -*- coding: cp936 -*-
import datetime
import json,csv,re,time
import pandas as pd
from pymongo import MongoClient


client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.cundan
collection = db.cundan





shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))
shijian=str(shijian)
shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian1=shijian-datetime.timedelta(days=20)




url='yinhang.csv'
df = pd.read_table(url,sep=',',encoding='GB18030')   #用dictreader根据行内容查找


for i in range(0,len(df)):    
        cursor = collection.find({"$and":[{'发行日':{'$gte':str(shijian1)}},{'发行人':{'$regex':df.loc[i,'银行']}}]}) 
        cun_df = pd.DataFrame(list(cursor))
        if cun_df.empty:

            print('11')
        else:
            cun_df1=cun_df[cun_df['期限']=='1月']
            cun_df1=cun_df1.reset_index(drop=False)

            if cun_df1.empty:
               print(111)
            else:
                df.loc[i,'1月利率']=cun_df1.loc[(len(cun_df1)-1),'收益率']
                df.loc[i,'1月发行日']=cun_df1.loc[(len(cun_df1)-1),'发行日']
                df.loc[i,'1月发行量']=cun_df1.loc[(len(cun_df1)-1),'计划发行(亿)']

            cun_df2=cun_df[cun_df['期限']=='3月']
            cun_df2=cun_df2.reset_index(drop=False)

            if cun_df2.empty:
               print(111)
            else:
                print(cun_df2)
                df.loc[i,'3月利率']=cun_df2.loc[(len(cun_df2)-1),'收益率']
                df.loc[i,'3月发行日']=cun_df2.loc[(len(cun_df2)-1),'发行日']
                df.loc[i,'3月发行量']=cun_df2.loc[(len(cun_df2)-1),'计划发行(亿)']
                
            cun_df3=cun_df[cun_df['期限']=='6月']
            cun_df3=cun_df3.reset_index(drop=False)

            if cun_df3.empty:
               print(111)
            else:
                df.loc[i,'6月利率']=cun_df3.loc[(len(cun_df3)-1),'收益率']
                df.loc[i,'6月发行日']=cun_df3.loc[(len(cun_df3)-1),'发行日']
                df.loc[i,'6月发行量']=cun_df3.loc[(len(cun_df3)-1),'计划发行(亿)']

            cun_df4=cun_df[cun_df['期限']=='9月']
            cun_df4=cun_df4.reset_index(drop=False)

            if cun_df4.empty:
               print(111)
            else:
                df.loc[i,'9月利率']=cun_df4.loc[(len(cun_df4)-1),'收益率']
                df.loc[i,'9月发行日']=cun_df4.loc[(len(cun_df4)-1),'发行日']
                df.loc[i,'9月发行量']=cun_df4.loc[(len(cun_df4)-1),'计划发行(亿)']



            cun_df5=cun_df[cun_df['期限']=='1年']
            cun_df5=cun_df5.reset_index(drop=False)

            if cun_df5.empty:
               print(111)
            else:
                df.loc[i,'1年利率']=cun_df5.loc[(len(cun_df5)-1),'收益率']
                df.loc[i,'1年发行日']=cun_df5.loc[(len(cun_df5)-1),'发行日']
                df.loc[i,'1年发行量']=cun_df5.loc[(len(cun_df5)-1),'计划发行(亿)']



#cursor = collection.find({ "$and":[{'发行日':'2018-03-06'}] })  

#df = pd.DataFrame(list(cursor))

#print(df)
df.to_csv('重要交易对手同业存单.csv',mode='w',header=True)


11
11
11
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5d5bdcda12e3440ba0f75b09      1  111904081  19中国银行CD081  报价发行   
1        3  5d5d0f3dcbbc884b20dedb6a    130  111904081  19中国银行CD081  报价发行   
2        4  5d5e606acbbc882314c0d743      2  111904083  19中国银行CD083  报价发行   
3        6  5d5fad6fa61b102370a3eee4      2  111904085  19中国银行CD085  报价发行   
4        8  5d5fad6fa61b102370a3efac    145  111904083  19中国银行CD083  报价发行   
5        9  5d63a2aaa61b103d8cd7ce1a      2  111904086  19中国银行CD086  报价发行   
6       11  5d63a2aaa61b103d8cd7cf41    103  111904085  19中国银行CD085  报价发行   
7       13  5d64fb9ca61b1035f8ebfae5    143  111904086  19中国银行CD086  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级  银行分类  
0  2019-08-21      NaN   零息  2.7000  3月  2019-08-20   --     61.0  AAA  中国银行  
1  2019/08/21     61.0   零息     2.7  3月  2019-08-21   --      NaN  AAA  中国银行  
2  2019-08-23      NaN   零息  2.7500  3月  2019-08-22  

111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5d679b54a61b101ad40ee81c     15  111919190  19恒丰银行CD190  报价发行   
1        1  5d68ee83a61b103418b91edc     57  111919190  19恒丰银行CD190  报价发行   
2        2  5d6e30b8a61b103560deae31     27  111919191  19恒丰银行CD191  报价发行   
3        3  5d6f7e6ea61b103b5c710ee7     20  111919192  19恒丰银行CD192  报价发行   
4        4  5d6f7e6ea61b103b5c710fe3    106  111919191  19恒丰银行CD191  报价发行   
5        5  5d70cfc5a61b100b843c6ffd    101  111919192  19恒丰银行CD192  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-08-30      NaN   零息  3.0200  3月  2019-08-29   --      5.0  AAA   城商  
1  2019-08-30      1.5   零息    3.02  3月  2019-08-30   --      NaN  AAA   城商  
2  2019-09-04      NaN   零息  3.0600  3月  2019-09-03   --      8.0  AAA   城商  
3  2019-09-05      NaN   零息  3.0600  3月  2019-09-04   --      7.0  AAA   城商  
4  2019-09-04      3.0   零息    3.06  3月  2019-09-04   --      NaN 

111
    level_0                       _id  index       债券代码          发行人  发行方式  \
0         0  5d5bdcda12e3440ba0f75b98    144  111985802  19达州银行CD148  报价发行   
1         4  5d5d0f3dcbbc884b20dedacf    127  111985934  19达州银行CD152  报价发行   
2         8  5d5d0f3dcbbc884b20dedafa     18  111985802  19达州银行CD148  报价发行   
3         9  5d5e606acbbc882314c0d7df    158  111986084  19达州银行CD156  报价发行   
4        13  5d5e606acbbc882314c0d818     19  111985934  19达州银行CD152  报价发行   
5        14  5d63a2aaa61b103d8cd7cecc    180  111986387  19达州银行CD164  报价发行   
6        18  5d64fb9ca61b1035f8ebfa3c    137  111986518  19达州银行CD168  报价发行   
7        24  5d66475ea61b102a8863d3d7    141  111986658  19达州银行CD174  报价发行   
8        26  5d679b54a61b101ad40ee86e     97  111986750  19达州银行CD176  报价发行   
9        31  5d68ee83a61b103418b91e91     53  111986801  19达州银行CD181  报价发行   
10       34  5d6cde9aa61b1032045f7caa    121  111986906  19达州银行CD184  报价发行   
11       38  5d6e30b8a61b103560deaeae    152  111987057  19达

111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0       10  5d66475ea61b102a8863d3a2     88  111986605  19贵阳银行CD109  报价发行   
1       15  5d679b54a61b101ad40ee89f     40  111986605  19贵阳银行CD109  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-08-29      NaN   零息  2.8300  3月  2019-08-28   --      5.0  AAA   城商  
1  2019-08-29      4.0   零息    2.83  3月  2019-08-29   --      NaN  AAA   城商  
111
111
111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0       17  5d679b54a61b101ad40ee866     89  111986742  19桂林银行CD190  报价发行   
1       22  5d68ee83a61b103418b91e98     60  111986808  19桂林银行CD192  报价发行   
2       23  5d68ee83a61b103418b91eab      8  111986742  19桂林银行CD190  报价发行   
3       24  5d6cde9aa61b1032045f7c9e    109  111986894  19桂林银行CD193  报价发行   
4       25  5d6cde9aa61b1032045f7ccc      7  111986808  19桂林银行CD192  报价发行   
5       29  5d6e30b8a61b103560deaed8     26  1119868

111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5d5d0f3dcbbc884b20dedab2     98  111985905  19晋商银行CD087  报价发行   
1        7  5d5e606acbbc882314c0d828     35  111985905  19晋商银行CD087  报价发行   

          发行日  实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-08-22      NaN   零息  3.1  3月  2019-08-21   --      4.2  AA+   城商  
1  2019-08-22      4.2   零息  3.1  3月  2019-08-22   --      NaN  AA+   城商  
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        2  5d6f7e6ea61b103b5c710f0b     56  111987106  19晋中银行CD018  报价发行   
1        3  5d70cfc5a61b100b843c6f16     54  111987237  19晋中银行CD019  报价发行   
2        5  5d70cfc5a61b100b843c6fe8     80  111987106  19晋中银行CD018  报价发行   
3        7  5d72282aa61b103e78249b7f     95  111987237  19晋中银行CD019  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-09-05      NaN   零息  3.7000  3月  2019-09-04   --      0.5  AA- 

111
111
   level_0                       _id  index       债券代码            发行人  发行方式  \
0       26  5d6cde9aa61b1032045f7c7c     75  111986860  19厦门国际银行CD176  报价发行   
1       27  5d6e30b8a61b103560deaeef     49  111986860  19厦门国际银行CD176  报价发行   
2       30  5d6f7e6ea61b103b5c710f3c    105  111987155  19厦门国际银行CD178  报价发行   
3       33  5d70cfc5a61b100b843c6f7e    158  111987341  19厦门国际银行CD181  报价发行   
4       37  5d70cfc5a61b100b843c6fc3     43  111987155  19厦门国际银行CD178  报价发行   
5       39  5d72282aa61b103e78249b15    147  111987490  19厦门国际银行CD183  报价发行   
6       41  5d72282aa61b103e78249b2c     12  111987341  19厦门国际银行CD181  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-09-03      NaN   零息    2.85  3月  2019-09-02   --      0.5  AAA   城商  
1  2019-09-03      0.5   零息    2.85  3月  2019-09-03   --      NaN  AAA   城商  
2  2019-09-05      NaN   零息  2.8700  3月  2019-09-04   --     14.0  AAA   城商  
3  2019-09-06      NaN   零息  2.8300  3月  2019-0

    level_0                       _id  index       债券代码          发行人  发行方式  \
0         1  5d5bdcda12e3440ba0f75b8d    133  111985791  19苏州银行CD232  报价发行   
1         3  5d5d0f3dcbbc884b20deda99     73  111985880  19苏州银行CD234  报价发行   
2         5  5d5d0f3dcbbc884b20dedb00     24  111985791  19苏州银行CD232  报价发行   
3         9  5d5e606acbbc882314c0d83e     57  111985880  19苏州银行CD234  报价发行   
4        13  5d63a2aaa61b103d8cd7ce67     79  111986286  19苏州银行CD240  报价发行   
5        14  5d63a2aaa61b103d8cd7cee5     11  111986222  19苏州银行CD238  报价发行   
6        18  5d64fb9ca61b1035f8ebfaaa     84  111986286  19苏州银行CD240  报价发行   
7        24  5d679b54a61b101ad40ee82d     32  111986685  19苏州银行CD245  报价发行   
8        26  5d68ee83a61b103418b91ed3     48  111986685  19苏州银行CD245  报价发行   
9        30  5d6e30b8a61b103560deae84    110  111987015  19苏州银行CD248  报价发行   
10       34  5d6f7e6ea61b103b5c710fa5     44  111987015  19苏州银行CD248  报价发行   

           发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行

111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        2  5d66475ea61b102a8863d36e     36  111986553  19营口银行CD075  报价发行   
1        3  5d679b54a61b101ad40ee8c6     79  111986553  19营口银行CD075  报价发行   

          发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-08-29      NaN   零息  3.5700  3月  2019-08-28   --      5.0  AA+   城商  
1  2019-08-29      3.0   零息    3.57  3月  2019-08-29   --      NaN  AA+   城商  
111
111
111
    level_0                       _id  index       债券代码           发行人  发行方式  \
0         0  5d5e606acbbc882314c0d7c8    135  111986061  19张家口银行CD037  报价发行   
1         1  5d5fad6fa61b102370a3ef48     45  111986061  19张家口银行CD037  报价发行   
2         2  5d63a2aaa61b103d8cd7ce68     80  111986287  19张家口银行CD038  报价发行   
3         3  5d63a2aaa61b103d8cd7ce69     81  111986288  19张家口银行CD039  报价发行   
4         4  5d64fb9ca61b1035f8ebf9ff     76  111986457  19张家口银行CD040  报价发行   
5         5  5d64fb9ca61b1035f8ebfaa8     82

111
111
11
11
11
11
111
   level_0                       _id  index       债券代码                发行人  \
0        4  5d63a2aaa61b103d8cd7ce9f    135  111986342  19江苏江南农村商业银行CD107   
1       11  5d64fb9ca61b1035f8ebfa7b     37  111986342  19江苏江南农村商业银行CD107   
2       21  5d6cde9aa61b1032045f7c88     87  111986872  19江苏江南农村商业银行CD116   
3       23  5d6e30b8a61b103560deaee6     40  111986872  19江苏江南农村商业银行CD116   
4       24  5d70cfc5a61b100b843c6f2d     77  111987260  19江苏江南农村商业银行CD117   
5       32  5d72282aa61b103e78249b6e     78  111987260  19江苏江南农村商业银行CD117   

   发行方式         发行日  实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级  \
0  报价发行  2019-08-27      NaN   零息     2.9  3月  2019-08-26   --      2.5  AAA   
1  报价发行  2019-08-27      2.5   零息     2.9  3月  2019-08-27   --      NaN  AAA   
2  报价发行  2019-09-03      NaN   零息    2.82  3月  2019-09-02   --      0.5  AAA   
3  报价发行  2019-09-03      0.5   零息    2.82  3月  2019-09-03   --      NaN  AAA   
4  报价发行  2019-09-06      NaN   零息  2

11
11
11
111
   level_0                       _id  index       债券代码           发行人  发行方式  \
0        0  5d64fb9ca61b1035f8ebf9f2     63  111986444  19中山农商行CD025  报价发行   

          发行日 实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-08-28    None   零息  2.9500  3月  2019-08-27   --      1.0  AA+  农商行  
111
111
111
    level_0                       _id  index       债券代码            发行人  发行方式  \
0         0  5d5bdcda12e3440ba0f75b6e    102  111985760  19重庆农村商行CD183  报价发行   
1         4  5d5d0f3dcbbc884b20dedb1b     51  111985760  19重庆农村商行CD183  报价发行   
2         6  5d5fad6fa61b102370a3ef07     37  111986147  19重庆农村商行CD186  报价发行   
3         7  5d63a2aaa61b103d8cd7cf21     71  111986147  19重庆农村商行CD186  报价发行   
4         8  5d66475ea61b102a8863d38d     67  111986584  19重庆农村商行CD187  报价发行   
5        11  5d679b54a61b101ad40ee863     86  111986739  19重庆农村商行CD190  报价发行   
6        16  5d679b54a61b101ad40ee8ae     55  111986584  19重庆农村商行CD187  报价发行   
7        18  5d68ee83a61

#  微信群价格

In [2]:
# -*- coding: cp936 -*-
import pandas as pd

import matplotlib.pyplot as plt
#from PIL import Image
import numpy as np

import jieba,os
import jieba.analyse
import json,csv,re,time,datetime
from pymongo import MongoClient
#from wordcloud import WordCloud
shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))

shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian=str(shijian.strftime("%Y-%m-%d"))

print(shijian)
shuru=input('请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:')
if shuru!='':
    shijian=shuru

client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.piaofen
collection = db.piaofen
cursor = collection.find({"$and":[{'time':shijian},{'hanglei2':1}]})
df = pd.DataFrame(list(cursor))

df = df.sort_values(by='time2', ascending=True)
df=df.drop_duplicates(['content'],keep='last')   #删除连接中的重复行

for i in range (1,len(df)):
      zhaop= re.search('[^0-9][0-9]\.[0-9]+[^元万亿]',str(df.loc[i,'content']))
      if zhaop:
         df.loc[i,'jiage']=1
      zhaop1= re.search('[0-9]+%',str(df.loc[i,'content']))
      if zhaop1:
         df.loc[i,'jiage']=1

for i in range (1,len(df)):   
      zhaop3= re.search('[^0-9][1-9][0-9][0-9][^0-9万天]',str(df.loc[i,'content']))
      if zhaop3:
             df.loc[i,'jiage']=1
      

df=df[df['jiage']==1]
df1=df[(df['shou']+df['chu']+df['shoudai']+df['chudai']+df['shouhui']+df['chuhui'])>0]
df1=df1.sort_values('hanglei1', ascending=True)  #吧银行类别排序

columns = ['time','time2','hanglei1','nickname','content']
df1.to_csv('票据价格.csv',mode='w',header=True,columns=columns)

df2=df[(df['shoufu']+df['chufu'])>0]
df2=df2.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df2.to_csv('福费廷价格.csv',mode='w',header=True,columns=columns)

df3=df[(df['shoucun']+df['chucun'])>0]
df3=df3.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df3.to_csv('资金价格.csv',mode='w',header=True,columns=columns)

df4=df[(df['shouli']+df['chuli'])>0]
df4=df4.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df4.to_csv('理财价格.csv',mode='w',header=True,columns=columns)


2019-06-27
请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:
